<a href="https://colab.research.google.com/github/google/neural-tangents/blob/main/notebooks/myrtle_kernel_with_neural_tangents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright 2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [0]:
!pip install -q --upgrade pip
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents

In [0]:
# Check which device you are running  before benchmarking
# For examples V100 can perform 40x double precision flops compared to P4
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

In [0]:
from jax.config import config
# Enable float64 for JAX
config.update("jax_enable_x64", True)

import jax.numpy as np
from jax import jit
import functools
from jax import random

import neural_tangents as nt
from neural_tangents import stax


In [0]:
key = random.PRNGKey(0)

In [0]:
# Network architecture described in 
# Shankar et al., Neural Kernels Without Tangents, 2020.
# https://arxiv.org/abs/2003.02237

def MyrtleNetwork(depth, W_std=np.sqrt(2.0), b_std=0.):
  layer_factor = {5: [2, 1, 1], 7: [2, 2, 2], 10: [3, 3, 3]}
  width = 1
  activation_fn = stax.Relu()
  layers = []
  conv = functools.partial(stax.Conv, W_std=W_std, b_std=b_std, padding='SAME')
  
  layers += [conv(width, (3, 3)), activation_fn] * layer_factor[depth][0]
  layers += [stax.AvgPool((2, 2), strides=(2, 2))]
  layers += [conv(width, (3, 3)), activation_fn] * layer_factor[depth][1]
  layers += [stax.AvgPool((2, 2), strides=(2, 2))]
  layers += [conv(width, (3, 3)), activation_fn] * layer_factor[depth][2]
  layers += [stax.AvgPool((2, 2), strides=(2, 2))] * 3

  layers += [stax.Flatten(), stax.Dense(10, W_std, b_std)]

  return stax.serial(*layers)

In [0]:
_, _, ker_fn = MyrtleNetwork(5)
ker_fn = jit(ker_fn, static_argnames='get')

In [0]:
key1, key2 = random.split(key)
input_x1 = random.normal(key1, shape=(10, 32, 32, 3))
input_x2 = random.normal(key2, shape=(10, 32, 32, 3))

In [8]:
# Note that library computes the full matrix thus lead to 2x inefficiency.
# With custom batching, only compute upper triangles.
# First loop during timing run includes jit compilation time and maybe slower.
%%timeit
kdd = ker_fn(input_x1, None, 'nngp').block_until_ready()

1 loops, best of 3: 35.7 ms per loop


In [9]:
%%timeit
ktd = ker_fn(input_x2, input_x1, 'nngp').block_until_ready()

1 loops, best of 3: 39.4 ms per loop


In [10]:
# Compute NNGP and NTK 
%%timeit
kdd = ker_fn(input_x1, None, ('nngp', 'ntk'))
kdd.nngp.block_until_ready()

1 loops, best of 3: 53.5 ms per loop


Using NT's batching function

In [0]:
_, _, ker_fn = MyrtleNetwork(5)
ker_fn = nt.batch(ker_fn, batch_size=10)

In [0]:
input_x = random.normal(key, shape=(50, 32, 32, 3))

In [13]:
# Note that library computes the full matrix thus lead to 2x inefficiency.
# With custom batching, only compute upper triangles.
# First loop during timing run includes jit compilation time and maybe slower.
%%timeit
kdd = ker_fn(input_x, None, 'nngp').block_until_ready()

1 loops, best of 3: 991 ms per loop


In [14]:
# Compute NNGP and NTK 
%%timeit
kdd = ker_fn(input_x, None, ('nngp', 'ntk'))
kdd.nngp.block_until_ready()

1 loops, best of 3: 1.44 s per loop
